# Sistema Integrado de Trading WDO

Este notebook demonstra o funcionamento completo do sistema.

In [8]:
# Configuração inicial
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

# Configura caminhos
PROJECT_ROOT = Path(os.path.dirname(os.getcwd()))  # Sobe um nível da pasta notebooks
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Carrega variáveis de ambiente
load_dotenv(dotenv_path=PROJECT_ROOT / '.env')

# Configura visualizações
try:
    plt.style.use('seaborn')
except:
    plt.style.use('default')
    
pd.set_option('display.max_columns', None)

print('Configuração concluída!')

Configuração concluída!


In [14]:
# Carregamento dos dados
from src.data.loaders.market_data_loader import MarketDataLoader

# Define caminho do banco
db_path = PROJECT_ROOT / 'src' / 'data' / 'database' / 'candles.db'
print(f'Tentando acessar banco em: {db_path}')

# Carrega dados
loader = MarketDataLoader(str(db_path))
data = loader.load_data()

print(f'Dados carregados: {len(data)} registros')
print('Primeiros registros:')
print(data.head())

Tentando acessar banco em: c:\Users\rlcp0\AI Office\mini-dolar-strategy-v3\src\data\database\candles.db
Dados carregados: 94538 registros
Primeiros registros:
                         open      high       low     close  tick_volume  \
time                                                                       
2024-03-22 12:57:00  5136.406  5137.435  5134.862  5135.376         1605   
2024-04-01 09:00:00  5158.485  5161.051  5157.971  5160.538         2486   
2024-04-01 09:01:00  5161.051  5162.591  5157.971  5160.025         4655   
2024-04-01 09:02:00  5160.538  5160.538  5157.971  5160.025         1717   
2024-04-01 09:03:00  5160.538  5162.078  5158.998  5162.078         1969   

                     spread  real_volume  
time                                      
2024-03-22 12:57:00       1         8314  
2024-04-01 09:00:00       1        11091  
2024-04-01 09:01:00       1        24795  
2024-04-01 09:02:00       1         9498  
2024-04-01 09:03:00       1         9934  


In [10]:
# Preparação das features
from src.ml.features.feature_engineering import FeatureEngineering

# Cria features técnicas
engineer = FeatureEngineering()
features = engineer.create_technical_features(data)

# Adiciona features temporais
features = engineer.create_temporal_features(features)

print('Features criadas:')
for col in features.columns:
    print(f'- {col}')

Features criadas:
- open
- high
- low
- close
- tick_volume
- spread
- real_volume
- returns
- returns_volatility
- sma_5
- sma_5_slope
- sma_10
- sma_10_slope
- sma_20
- sma_20_slope
- sma_50
- sma_50_slope
- high_low_range
- daily_range
- volume_ma
- volume_ratio
- momentum
- acceleration
- hour
- minute
- dayofweek
- is_morning
- is_afternoon
- is_market_open
- is_open_hour
- is_close_hour
- is_lunch_time


In [11]:
from pathlib import Path
import yaml

# Caminho correto para o arquivo base_config.yaml
config_path = Path(r"C:\Users\rlcp0\AI Office\mini-dolar-strategy-v3\config\base_config.yaml")

# Verifica se o arquivo existe antes de tentar carregá-lo
if not config_path.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {config_path}")

# Carregar as configurações
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

print("Configurações Carregadas do YAML:")
print(config)

Configurações Carregadas do YAML:
{'market': {'symbol': 'WDO', 'timeframe': '1m', 'session_start': '09:00', 'session_end': '17:00', 'min_volume': 100}, 'technical': {'moving_averages': [9, 21, 50], 'rsi_period': 14, 'atr_period': 14, 'volatility_window': 20}, 'fundamental': {'news_impact_window': 60, 'economic_data_window': 5, 'sentiment_threshold': 0.5}, 'ml': {'sequence_length': 60, 'train_test_split': 0.2, 'validation_size': 0.1, 'batch_size': 32, 'epochs': 100, 'ensemble': {'n_estimators': 10, 'max_samples': 0.8, 'max_features': 0.8}}, 'trading': {'initial_capital': 100000, 'position_size': 1, 'max_positions': 4, 'stop_loss': 1.5, 'take_profit': 2.0, 'max_daily_loss': -2000, 'max_drawdown': -0.00233}, 'paths': {'data': 'data/', 'models': 'models/', 'reports': 'reports/', 'logs': 'logs/'}}


In [12]:
# Configuração dos agentes
from src.agents.market_agent import RiskManagementAgent, MarketRegimeAgent, FeatureSelectionAgent
from src.agents.agent_coordinator import AgentCoordinator

# Inicializa agentes
risk_agent = RiskManagementAgent(
    max_drawdown=config['trading']['max_drawdown']
)
regime_agent = MarketRegimeAgent(
    window_size=config['technical']['volatility_window']
)
feature_agent = FeatureSelectionAgent(
    initial_features=features.columns.tolist()
)

# Configura coordenador
coordinator = AgentCoordinator([
    risk_agent,
    regime_agent,
    feature_agent
])

print('Agentes configurados!')

Agentes configurados!


In [13]:
print("Parâmetros do YAML Carregados:")
print(config)

print("Configuração do Risk Agent:")
print(f"Max Drawdown: {risk_agent.max_drawdown}")

print("Configuração do Regime Agent:")
print(f"Window Size: {regime_agent.window_size}")

Parâmetros do YAML Carregados:
{'market': {'symbol': 'WDO', 'timeframe': '1m', 'session_start': '09:00', 'session_end': '17:00', 'min_volume': 100}, 'technical': {'moving_averages': [9, 21, 50], 'rsi_period': 14, 'atr_period': 14, 'volatility_window': 20}, 'fundamental': {'news_impact_window': 60, 'economic_data_window': 5, 'sentiment_threshold': 0.5}, 'ml': {'sequence_length': 60, 'train_test_split': 0.2, 'validation_size': 0.1, 'batch_size': 32, 'epochs': 100, 'ensemble': {'n_estimators': 10, 'max_samples': 0.8, 'max_features': 0.8}}, 'trading': {'initial_capital': 100000, 'position_size': 1, 'max_positions': 4, 'stop_loss': 1.5, 'take_profit': 2.0, 'max_daily_loss': -2000, 'max_drawdown': -0.00233}, 'paths': {'data': 'data/', 'models': 'models/', 'reports': 'reports/', 'logs': 'logs/'}}
Configuração do Risk Agent:
Max Drawdown: -0.00233
Configuração do Regime Agent:
Window Size: 20
